# Resources 
* Google maps API [https://github.com/googlemaps/google-maps-services-python]
* Earth Data Science: https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-raster-plots/interactive-maps/ 
* Blog : https://medium.com/omdena/using-geospatial-data-analytics-a-friendly-guide-to-folium-and-rasterio-5568e724c699
* Load layers as Numpy arrays https://gis.stackexchange.com/questions/350771/earth-engine-simplest-way-to-move-from-ee-image-to-array-for-use-in-sklearn

# Note:
* **The Layer on top of folium basemaps will disappear after a few days. Please re-run the notebook to see the Images with your authentication**

In [1]:
import ee 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
import folium
import pickle
import pandas as pd
import IPython.display as disp

import io
from PIL import Image


%matplotlib inline

In [2]:

def get_coordinates(address,return_json=False,return_bb= False):
    '''Get lat-long for a given address
    SRC : https://stackoverflow.com/questions/25888396/how-to-get-latitude-longitude-with-python'''
    
    import requests
    from urllib import parse
    
    response = requests.get('https://nominatim.openstreetmap.org/search/' + parse.quote(address) + '?format=json')

    resp= response.json()
    
    #return complete response file 
    if return_json:
        return resp
    
    if return_bb:
        return [(float(resp[0]['boundingbox'][i]),float(resp[0]['boundingbox'][i+2])) for i in range(len(resp[0]['boundingbox'])//2)]
    
#   'returns LAT -  LONG'
    
    return (float(resp[0]['lat']),float(resp[0]['lon']))
    

In [3]:
# setup to add ee layer to folium

def add_ee_layer(self, ee_image_object, vis_params, name,opacity=0.90):
    '''function to add GEE layer to folium,
    Source : GEE'''
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
          tiles=map_id_dict['tile_fetcher'].url_format,
          attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
          name=name,
          opacity=opacity,
          overlay=True,
          control=True
      ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


def get_best_image(image_collection,
                   year,
                   area_of_interest,
                   month=1):
    '''filter the best image( with the least cloud cover value) for the given year and given month'''

    start_date = ee.Date.fromYMD(**{
        'day':1 ,
        'month': month,
        'year' : year})
    end_date = ee.Date.fromYMD(**{
        'day':30 ,
        'month': month,
        'year' : year})
    
    print(image_collection.filterDate(start_date,end_date).size().getInfo())
    
    if not area_of_interest:
        # sort image by asc order of cloud pixel values
        img = image_collection.filterDate(start_date,end_date).sort('CLOUD_COVER').first()
    else:
        img = image_collection.filterDate(start_date,end_date).sort('CLOUD_COVER').first().clip(area_of_interest)
        
    
    return img 

**Authenticate and Initialize Google Earth Engine**
* **Running the ee.Authenticate() method generates a link, which is used to create a token for connecting to your google earth engine account.(If you don't have a Earth Engine account, you will have to sign up on the platform).After you sign in with your google account, you will get a token, which has to be pasted in the box generated by ee.Authenticate().** 
* **This completes the Authentication. Note that on Kaggle, you will have to generate a new Token for authenticating earth engine after a runtime a restarted(Atleast I know it this way.If anyone knows the right way, please correct me).**
* **After Authenticating earth engine, you will have to Initialize Earth Engine with ee.Initialize(). Now you are good to go.**

In [5]:
#authenticate earth engine
# ee.Authenticate()

In [6]:
# inintialize earth engine

ee.Initialize()

# Dataset Description
**Monthly average radiance composite images using nighttime data from the Visible Infrared Imaging Radiometer Suite (VIIRS) Day/Night Band (DNB).**

**Prior to averaging, the DNB data is filtered to exclude data impacted by stray light, lightning, lunar illumination, and cloud-cover.**

**Source :** [ https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMCFG#description ]

In [7]:
#VIIRS 
viirs_nightttime = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")
#dmps
# dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")

# international boundaries dataset
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

In [20]:

country_list = ['India','USA','Germany','China','Russia','Japan']


# get lat-long coordinates for above countries
coord_dict = {name:get_coordinates(name) for name in country_list}

#visualizing params
nighttimeVis = {'min': 0.0, 
                'max': 45}

# **India**

In [21]:
# getting a boundry 
india = countries.filter(ee.Filter.eq('country_na','India'))

# get best quality image (with lowest cloud cver), with a cliped image in area of interest
In_im = get_best_image(viirs_nightttime,year = 2020,area_of_interest=india,month = 11).select('avg_rad')
In_im2 = get_best_image(viirs_nightttime,year = 2012,area_of_interest=india,month = 11).select('avg_rad')


M1 = folium.Map(location  = coord_dict['India'] ,min_zoom = 4,zoom_start= 6)


M1.add_ee_layer(In_im, nighttimeVis, 'Nighttime 2020')
M1.add_ee_layer(In_im2, nighttimeVis, 'Nighttime 2012')
M1.add_child(folium.LayerControl())

M1

1
1


**Finding regions with increased growth(By subtracting the first raster (2020) with sceond raster (2012))**

In [22]:
dif1 = In_im.subtract(In_im2)

M2 = folium.Map(location  = coord_dict['India'] ,min_zoom = 4,zoom_start= 6 )


M2.add_ee_layer(dif1, nighttimeVis, 'Regions of New Growth(12-20)')
M2.add_child(folium.LayerControl())
display(M2)

**We can see growth around the outskirts of the major economical cities in India**

# USA 



In [23]:
# get boundry and clipped image
usa = countries.filter(ee.Filter.eq('country_na','United States'))
US_im = get_best_image(viirs_nightttime,year = 2020,area_of_interest=usa,month = 12).select('avg_rad')
US_im2 = get_best_image(viirs_nightttime,year = 2012,area_of_interest=usa,month = 12).select('avg_rad')


M3 = folium.Map(location  = coord_dict['USA'] ,min_zoom = 4,zoom_start= 5)


M3.add_ee_layer(US_im, nighttimeVis, 'Nighttime 2020')
M3.add_ee_layer(US_im2, nighttimeVis, 'Nighttime 2012')
M3.add_child(folium.LayerControl())
display(M3)

1
1


In [25]:
dif2 = US_im.subtract(US_im2)

M4 = folium.Map(location  = coord_dict['USA'] ,min_zoom = 4,zoom_start= 6 )


M4.add_ee_layer(dif2, nighttimeVis, 'Regions of New Growth(12-20)')
M4.add_child(folium.LayerControl())
display(M4)

# **Japan**

In [26]:

# japan image 

# a radius of 1000 kms around returned coordinates
japan = countries.filter(ee.Filter.eq('country_na','Japan'))
Jpn_im = get_best_image(viirs_nightttime,year = 2020,area_of_interest=japan,month = 12).select('avg_rad')
Jpn_im2 = get_best_image(viirs_nightttime,year = 2012,area_of_interest=japan,month = 12).select('avg_rad')



M5 = folium.Map(location  = coord_dict['Japan'] ,min_zoom = 6,zoom_start= 7)


M5.add_ee_layer(Jpn_im, nighttimeVis, 'Nighttime 2020')
M5.add_ee_layer(Jpn_im2, nighttimeVis, 'Nighttime 2012')
M5.add_child(folium.LayerControl())
M5

1
1


**The above visualization clearly shows that tokyo is one of the largest urban centers**

In [27]:
dif3 = Jpn_im.subtract(Jpn_im2)

M6 = folium.Map(location  = coord_dict['Japan'] ,min_zoom = 4,zoom_start= 6 )


M6.add_ee_layer(dif3, nighttimeVis, 'Regions of New Growth(12-20)')
M6.add_child(folium.LayerControl())
display(M6)

# **Germany**

In [28]:
# getting boundaries and clipped image
ger = countries.filter(ee.Filter.eq('country_na','Germany'))
Ger_im = get_best_image(viirs_nightttime,
                        year = 2020,
                        area_of_interest=ger
                        ,month = 12).select('avg_rad')
Ger_im2 = get_best_image(viirs_nightttime,
                        year = 2012,
                        area_of_interest=ger
                        ,month = 12).select('avg_rad')


M7 = folium.Map(location  = coord_dict['Germany'] ,min_zoom = 6,zoom_start= 7)


M7.add_ee_layer(Ger_im, nighttimeVis, 'Nighttime 2020')
M7.add_ee_layer(Ger_im2, nighttimeVis, 'Nighttime 2012')
M7.add_child(folium.LayerControl())
display(M7)

1
1


In [29]:
dif4 = Ger_im.subtract(Ger_im2)

M8= folium.Map(location  = coord_dict['Germany'] ,min_zoom = 4,zoom_start= 6 )


M8.add_ee_layer(dif4, nighttimeVis, 'Regions of New Growth(12-20)')
M8.add_child(folium.LayerControl())
display(M8)

# China

In [30]:
# getting boundaries and clipped image
chi = countries.filter(ee.Filter.eq('country_na','China'))
Chi_im = get_best_image(viirs_nightttime,
                        year = 2020,
                        area_of_interest=chi
                        ,month = 12).select('avg_rad')
Chi_im2 = get_best_image(viirs_nightttime,
                        year = 2012,
                        area_of_interest=chi
                        ,month = 12).select('avg_rad')


M9 = folium.Map(location  = coord_dict['China'] ,min_zoom = 5,zoom_start= 6)


M9.add_ee_layer(Chi_im, nighttimeVis, 'Nighttime 2020')
M9.add_ee_layer(Chi_im2, nighttimeVis, 'Nighttime 2012')
M9.add_child(folium.LayerControl())
display(M9)

1
1


In [32]:
dif5 = Chi_im.subtract(Chi_im2)

M10 = folium.Map(location  = coord_dict['China'] ,min_zoom = 4,zoom_start= 6 )


M10.add_ee_layer(dif5, nighttimeVis, 'Regions of New Growth(12-20)')
M10.add_child(folium.LayerControl())
display(M10)

# **Russia**

In [33]:
# getting boundaries and clipped image
rus= countries.filter(ee.Filter.eq('country_na','Russia'))
Rus_im = get_best_image(viirs_nightttime,
                        year = 2020,
                        area_of_interest=rus
                        ,month = 12).select('avg_rad')

Rus_im2 = get_best_image(viirs_nightttime,
                        year = 2012,
                        area_of_interest=rus
                        ,month = 12).select('avg_rad')

M11= folium.Map(location  = coord_dict['Russia'] ,min_zoom = 5,zoom_start= 6)


M11.add_ee_layer(Rus_im, nighttimeVis, 'Nighttime 2020')
M11.add_ee_layer(Rus_im2, nighttimeVis, 'Nighttime 2012')
M11.add_child(folium.LayerControl())
display(M11)

1
1
